In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
import plotly.express as px
from numpy.linalg import eigvalsh
from plotly.subplots import make_subplots
import torch

from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior
from dataset_sabr import generate_sabr_call_options
from dupire_pinn_trainer import DupirePINNTrainer

In [2]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

# Define the SABR model parameters
alpha = 0.20  # Stochastic volatility parameter
beta = 0.50   # Elasticity parameter
rho = -0.75   # Correlation between asset price and volatility
nu = 1.0      # Volatility of volatility parameter

# Other model parameters
risk_free_rate = np.log(1.02)  # Risk-free interest rate
underlying_price = 1.0         # Current price of the underlying asset

# Generate the dataset using the SABR model and Black-Scholes formula
call_option_dataset = generate_sabr_call_options(
    alpha=alpha,
    beta=beta,
    rho=rho,
    nu=nu,
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    risk_free_rate=risk_free_rate,
    underlying_price=underlying_price
)

# Maturity times and strike prices from the previous product grid setup
hypothetical_maturity_time_list = np.logspace(np.log10(0.01), np.log10(3.1), 100)
hypothetical_strike_price_list = np.logspace(np.log10(0.7), np.log10(1.75), 100)

# Create the product grid of maturity times and strike prices
hypothetical_product_grid = list(product(hypothetical_maturity_time_list, hypothetical_strike_price_list))
hypothetical_maturity_times, hypothetical_strike_prices = zip(*hypothetical_product_grid)
hypothetical_maturity_times, hypothetical_strike_prices = np.array(hypothetical_maturity_times), np.array(hypothetical_strike_prices)

# Reshape the data for 3D surface plotting
hypothetical_maturities_grid = hypothetical_maturity_times.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))  
hypothetical_strikes_grid = hypothetical_strike_prices.reshape((len(hypothetical_maturity_time_list), len(hypothetical_strike_price_list)))

In [3]:
n_roots = 350
smoothness_controller = 3.274549162877732e-05

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

# The constant_volatility is set to a reasonable value
constant_volatility = RBFVolatilitySurface.calculate_constant_volatility(
    call_option_dataset["Implied Volatility"],
    call_option_dataset["Time to Maturity"],
    call_option_dataset["Strike Price"],
    risk_free_rate,
    underlying_price
)

sampled_surface_coefficients = smoothness_prior.sample_smooth_surfaces(100)

In [4]:
# Loop through the sampled coefficients 
sampled_volatilities = []
for coefficients in sampled_surface_coefficients:
    
    # Initialize the RBFVolatilitySurface class for each set of coefficients
    rbf_surface = RBFVolatilitySurface(
        coefficients=coefficients,
        maturity_times=maturity_times,
        strike_prices=strike_prices,
        maturity_std=maturity_std,
        strike_std=strike_std,
        constant_volatility=constant_volatility
    )

    # Generate the volatility surface over the product grid of times and strikes
    surface_volatilities = [
        rbf_surface.implied_volatility_surface(T, K)
        for T, K in product_grid
    ]
    sampled_volatilities.extend(surface_volatilities)

In [5]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [6]:
hidden_dim = 128
n_layers = 4
batch_size = 100
pde_loss_coefficient = 1.0
maturity_zero_loss_coefficient = 100
strike_zero_loss_coefficient = 100
strike_infinity_loss_coefficient = 100
pre_train_learning_rate = 1e-3
fine_tune_learning_rate = 1e-4
pre_train_epochs = 50
fine_tune_epochs = 20
maturity_min = maturity_time_list.min()
maturity_max = maturity_time_list.max()
strike_min = strike_price_list.min()
strike_max = strike_price_list.max()
volatility_mean = np.mean(sampled_volatilities)
volatility_std = np.std(sampled_volatilities)
constant_volatility = False
strike_infinity = 2.5
device = 'cpu'

# Initialize the DupirePINNTrainer class
trainer = DupirePINNTrainer(
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    batch_size=batch_size,
    pde_loss_coefficient=pde_loss_coefficient,
    maturity_zero_loss_coefficient=maturity_zero_loss_coefficient,
    strike_zero_loss_coefficient=strike_zero_loss_coefficient,
    strike_infinity_loss_coefficient=strike_infinity_loss_coefficient,
    pre_train_learning_rate=pre_train_learning_rate,
    fine_tune_learning_rate=fine_tune_learning_rate,
    pre_train_epochs=pre_train_epochs,
    fine_tune_epochs=fine_tune_epochs,
    maturity_min=maturity_min,
    maturity_max=maturity_max,
    strike_min=strike_min,
    strike_max=strike_max,
    volatility_mean=volatility_mean,
    volatility_std=volatility_std,
    maturity_time_list=maturity_time_list,
    strike_price_list=strike_price_list,
    strike_std=strike_std,
    maturity_std=maturity_std,
    constant_volatility=constant_volatility,
    strike_infinity=strike_infinity,
    device=device
)

In [7]:
trainer.pre_train(
    sampled_surface_coefficients=sampled_surface_coefficients,
    experiment_name='initial test'
)

torch.Size([130])
torch.Size([130, 10])
tensor([0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200, 0.0200,
        0.0200, 0.0800, 0.0800, 0.0800, 0.0800, 0.0800, 0.0800, 0.0800, 0.0800,
        0.0800, 0.0800, 0.1700, 0.1700, 0.1700, 0.1700, 0.1700, 0.1700, 0.1700,
        0.1700, 0.1700, 0.1700, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500, 0.2500,
        0.2500, 0.2500, 0.2500, 0.2500, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.7500, 0.7500, 0.7500, 0.7500,
        0.7500, 0.7500, 0.7500, 0.7500, 0.7500, 0.7500, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.5000, 1.5000,
        1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 1.5000, 2.0000,
        2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000,
        3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000, 3.0000,
        3.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,


RuntimeError: size mismatch, got input (130), mat (130x10), vec (100)